# Hotel Review Classification

### Team 1: Renetta Nelson, Jacqueline Urenda

### June 26, 2023

## Importing Libraries

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv
import time
import random


## Retrieving Data from Website

In [3]:
url = ('https://www.tripadvisor.in/Hotels-g60750-San_Diego_California-Hotels.html')

user_agent = ({'User-Agent':
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                AppleWebKit/537.36 (KHTML, like Gecko) \
                Chrome/90.0.4430.212 Safari/537.36',
                'Accept-Language': 'en-US, en;q=0.5'})

page = requests.get(url, headers = user_agent)
soup = BeautifulSoup(page.text, 'html.parser')

# Find and extract the data elements.
hotels = []
for name in soup.findAll('div',{'class':'listing_title'}):
    hotels.append(name.text.strip())
ratings = []
for rating in soup.findAll('a',{'class':'ui_bubble_rating'}):
    ratings.append(rating['alt'])  
reviews = []
for review in soup.findAll('a',{'class':'review_count'}):
    reviews.append(review.text.strip())
prices = []
for p in soup.findAll('div',{'class':'price-wrap'}):
    prices.append(p.text.replace('₹','').strip())  
# Create the dictionary.

dict = {'Hotel Names':hotels,'Ratings':ratings,'Number of Reviews':reviews,'Prices':prices}



In [17]:
print(dict)

{'Hotel Names': ['1. Bahia Resort Hotel', '1. Bahia Resort Hotel', '2. Manchester Grand Hyatt San Diego', '2. Manchester Grand Hyatt San Diego', '3. Embassy Suites by Hilton San Diego Bay Downtown', '3. Embassy Suites by Hilton San Diego Bay Downtown', '4. San Diego Mission Bay Resort', '4. San Diego Mission Bay Resort', '5. Paradise Point Resort & Spa', '5. Paradise Point Resort & Spa', '6. Catamaran Resort Hotel and Spa', '6. Catamaran Resort Hotel and Spa', '7. San Diego Marriott La Jolla', '7. San Diego Marriott La Jolla', '8. Best Western Plus Island Palms Hotel & Marina', '8. Best Western Plus Island Palms Hotel & Marina', '9. Old Town Inn', '9. Old Town Inn', '10. Urban Boutique Hotel', '10. Urban Boutique Hotel', '11. Pacific Terrace Hotel', '11. Pacific Terrace Hotel', '12. Pantai Inn', '12. Pantai Inn', '13. Marriott Marquis San Diego Marina', '13. Marriott Marquis San Diego Marina', '14. Kings Inn San Diego', '14. Kings Inn San Diego', '15. Pendry San Diego', '15. Pendry San

Extracting Reviews Text Data:

In [5]:
#Extracting review data:
url = ('https://www.tripadvisor.in/Hotels-g60750-San_Diego_California-Hotels.html')

user_agent = ({'User-Agent':
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                AppleWebKit/537.36 (KHTML, like Gecko) \
                Chrome/90.0.4430.212 Safari/537.36',
                'Accept-Language': 'en-US, en;q=0.5'})


html = requests.get(url, headers = user_agent)

html.status_code

bsobj = BeautifulSoup(html.content, 'html.parser')


In [6]:
links = []
for review in bsobj.findAll('a',{'class': 'review_count'}):
    a = review['href']
    a = 'https://www.tripadvisor.in'+ a

    print(a)

    print(a.find('Reviews'))
    a= a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
    print(a)

    links.append(a)

    links

https://www.tripadvisor.in/Hotel_Review-g60750-d83361-Reviews-Bahia_Resort_Hotel-San_Diego_California.html#REVIEWS
54
https://www.tripadvisor.in/Hotel_Review-g60750-d83361-Reviews-or{}-Bahia_Resort_Hotel-San_Diego_California.html#REVIEWS
https://www.tripadvisor.in/Hotel_Review-g60750-d80219-Reviews-Manchester_Grand_Hyatt_San_Diego-San_Diego_California.html#REVIEWS
54
https://www.tripadvisor.in/Hotel_Review-g60750-d80219-Reviews-or{}-Manchester_Grand_Hyatt_San_Diego-San_Diego_California.html#REVIEWS
https://www.tripadvisor.in/Hotel_Review-g60750-d80109-Reviews-Embassy_Suites_by_Hilton_San_Diego_Bay_Downtown-San_Diego_California.html#REVIEWS
54
https://www.tripadvisor.in/Hotel_Review-g60750-d80109-Reviews-or{}-Embassy_Suites_by_Hilton_San_Diego_Bay_Downtown-San_Diego_California.html#REVIEWS
https://www.tripadvisor.in/Hotel_Review-g60750-d119721-Reviews-San_Diego_Mission_Bay_Resort-San_Diego_California.html#REVIEWS
55
https://www.tripadvisor.in/Hotel_Review-g60750-d119721-Reviews-or{}-San

In [9]:
from random import randint
from time import sleep

reviews = []

for link in links:
    d=[5,10,15,20,25]
    html2 = requests.get(link.format(i for i in d), headers= user_agent)
    sleep(randint(1,5))
    bsobj2 = BeautifulSoup(html2.content, 'html.parser')
    for r in bsobj2.findAll('span',{'class': 'MInAm _a'}):
        reviews.append(r.span.text.strip())
        print(r.span.text.strip())
reviews

Dear Brianna, Happy Anniversary! Thank you for spending it at the Bahia Resort Hotel. It's a pleasure to hear that you had a fun and relaxing stay and enjoyed our staff's friendly service. We look forward to welcoming you back soon.
Dear Bahia Resort Hotel guest, thank you for your continued visits, wonderful 5 star review, and recommendation. We believe that hospitality is personal and it all starts with a dedicated and caring team. Your feedback makes us proud, and Heather and staff will certainly appreciate your recognition. Furthermore, we appreciate your acknowledgement of our commitment to continually enhance our resort to provide the best possible experience for our guests. Thanks again, we look forward to your return visit in the near future.
Dear guest, we appreciate hearing from you. The Bahia Resort Hotel is known for its exceptional guest services. It seems Renee hit the mark with your recent room booking, and we love hearing about it. It's great to hear that you are excite

["Dear Brianna, Happy Anniversary! Thank you for spending it at the Bahia Resort Hotel. It's a pleasure to hear that you had a fun and relaxing stay and enjoyed our staff's friendly service. We look forward to welcoming you back soon.",
 'Dear Bahia Resort Hotel guest, thank you for your continued visits, wonderful 5 star review, and recommendation. We believe that hospitality is personal and it all starts with a dedicated and caring team. Your feedback makes us proud, and Heather and staff will certainly appreciate your recognition. Furthermore, we appreciate your acknowledgement of our commitment to continually enhance our resort to provide the best possible experience for our guests. Thanks again, we look forward to your return visit in the near future.',
 "Dear guest, we appreciate hearing from you. The Bahia Resort Hotel is known for its exceptional guest services. It seems Renee hit the mark with your recent room booking, and we love hearing about it. It's great to hear that you 

In [15]:
reviews_df = pd.DataFrame(reviews, columns=['review_text'])

reviews_df.head(10)

,review_text
0,"Dear Brianna, Happy Anniversary! Thank you for..."
1,"Dear Bahia Resort Hotel guest, thank you for y..."
2,"Dear guest, we appreciate hearing from you. Th..."
3,"Dear guest, we appreciate hearing from you. Th..."
4,"Dear Angela, it's a pleasure to read your deta..."
5,"Dear valued guest, it's a pleasure to hear abo..."
6,"Dear Bryan, thank you for booking your stay wi..."
7,"Dear Jennifer, thank you for sharing your grea..."
8,"Dear Leslie, thank you for your continued visi..."
9,"Hello nidhi457, Thank you for leaving feedbac..."


In [16]:
reviews_df.shape

(174, 1)

## Extracting the Data

In [3]:
# Initialize lists

hotel_names = []
hotel_ratings = []
hotel_reviews = []
hotel_prices = []

# Hotel Data

h_content = BeautifulSoup(hotels, 'html.parser')

for hotel_name in BeautifulSoup.find_all('div', {'class': 'listing_title'}):
    hotel_names.append(hotel_name.text)

for hotel_rating in BeautifulSoup.find_all('a', {'class': 'ui_bubble_raing'}):
    hotel_ratings.append(hotel_rating['alt'])

for hotel_review in BeautifulSoup.find_all('a', {'class': 'review_count'}):
    hotel_reviews.append(hotel_review.text)

for hotel_price in BeautifulSoup.find_all('div', {'class': 'price-wrap'}):
    hotel_prices.append(hotel_price.text)


AttributeError: 'str' object has no attribute 'descendants'

## Loading Data 

In [ ]:

hotels_df = pd.DataFrame()

# Load data into dataframe

hotels_df["name"] = hotel_names
hotels_df["review_amt"] = hotel_reviews
hotels_df["ratings"] = hotel_ratings
hotels_df["costs"] = hotel_prices

hotels_df.head(20)


## Data Preparation